In [40]:
import os

import numpy as np
import json
import pandas as pd

import matplotlib
%matplotlib inline
from matplotlib import pylab as plt

from collections import Counter, defaultdict
from scipy.interpolate import spline
import scipy.stats

In [41]:
results_dir = './results/'

In [42]:
def count_emotions(result):
    blob = [x['emotion'] for x in result if len(x) > 0]
    return defaultdict(lambda: 0, Counter(blob))

In [43]:
def happines_by_time(result):
    def extract_happiness_score(x):
        if len(x) > 0:
            if x['emotion'] == 'happy': return 1
            elif x['emotion'] == 'sad': return -1

        return 0

    xs = np.arange(len(result))
    ys = np.array(list(map(extract_happiness_score, result)))

#     xs = [xs[i] for i in range(0, len(xs), 20)]
#     ys = [ys[i] for i in range(0, len(ys), 20)]

#     x_old = xs
#     x_new1 = np.linspace(0, xs[-1], 100)
#     x_new2 = np.linspace(0, xs[-1], 1000)
#     smooth = spline(x_old, ys, x_new1)
#     smooth = spline(x_new1, smooth, x_new2)

    return xs, ys

In [44]:
imdb_df = pd.read_csv('./imdb_250.csv', index_col='index')

In [45]:
rating = {}
for result_name in os.listdir(results_dir):
    result_path = results_dir + result_name
    result_id = int(result_name[:-5])
    
    with open(result_path) as fin:
        result = json.load(fin)
        
    emotion_counter = count_emotions(result)
    rating[result_id] = emotion_counter
    
#     happy_xs, happy_ys = happines_by_time(result)
#     happy_smoothed_ys = window_smooth(happy_ys, 10)
#     ys = happy_smoothed_ys[happy_smoothed_ys != 0]
#     xs = happy_xs[happy_smoothed_ys != 0]
    
#     print(result_id, imdb_df.loc[result_id]['title'], imdb_df.loc[result_id]['genre'])
#     xnew = np.linspace(xs[0], xs[-1], 300)
#     power_smooth = spline(xs, ys, xnew)
#     plt.plot(xnew, power_smooth)
#     plt.ylim(-1.2, 1.2)
#     plt.show()

In [46]:
def window_smooth(signal, window_size):
    pad_width = window_size // 2
    padded_signal = np.lib.pad(signal, pad_width=pad_width, mode='constant', constant_values=(0, 0))
    
    smoothed_signal = []
    for i in range(pad_width, len(signal) + pad_width):
        smoothed_signal.append(scipy.stats.mode(padded_signal[i - pad_width:i + pad_width + 1])[0][0])
        
    return np.array(smoothed_signal)

In [47]:
rating_list = list(rating.items())
# rating_sorted = sorted(rating_list, key=lambda x: x[1]['sad'] / sum(x[1].values()))
rating_sorted = sorted(rating_list, key=lambda x: x[1]['happy'] / (x[1]['sad'] + x[1]['happy']))
# rating_sorted = sorted(rating_list, key=lambda x: sum(x[1].values()))
for x in rating_sorted:
    print(x[0], imdb_df.loc[x[0]]['title'], imdb_df.loc[x[0]]['genre'])

50 Apocalypse Now Drama, War
89 To Kill a Mockingbird Crime, Drama


KeyError: 'the label [253] is not in the [index]'

In [48]:
emotions = []
for i, d in rating_list:
    emotions.append({
        'index': i,
        'h/(h+s)': d['happy'] / (d['happy'] + d['sad']),
        'h/all': d['happy'] / sum(d.values()),
        's/all': d['sad'] / sum(d.values())
    })

In [53]:
df = pd.DataFrame(emotions)
# df = df.set_index('index')
df.to_csv('emotions.csv')

In [55]:
for x in df.to_dict(orient='records'):
    print(x)

{'h/(h+s)': 0.30094043887147337, 's/all': 0.30886426592797783, 'h/all': 0.1329639889196676, 'index': 1.0}
{'h/(h+s)': 0.12450592885375494, 's/all': 0.53245192307692313, 'h/all': 0.075721153846153841, 'index': 2.0}
{'h/(h+s)': 0.065934065934065936, 's/all': 0.69672131147540983, 'h/all': 0.049180327868852458, 'index': 3.0}
{'h/(h+s)': 0.13793103448275862, 's/all': 0.4838709677419355, 'h/all': 0.077419354838709681, 'index': 4.0}
{'h/(h+s)': 0.13278008298755187, 's/all': 0.31618759455370649, 'h/all': 0.048411497730711045, 'index': 5.0}
{'h/(h+s)': 0.26285714285714284, 's/all': 0.39938080495356038, 'h/all': 0.14241486068111456, 'index': 6.0}
{'h/(h+s)': 0.2140221402214022, 's/all': 0.31415929203539822, 'h/all': 0.085545722713864306, 'index': 7.0}
{'h/(h+s)': 0.18227848101265823, 's/all': 0.38044758539458184, 'h/all': 0.084805653710247356, 'index': 8.0}
{'h/(h+s)': 0.28494623655913981, 's/all': 0.28663793103448276, 'h/all': 0.11422413793103449, 'index': 9.0}
{'h/(h+s)': 0.28865979381443296, 

In [ ]:
emotions